# Engenharia de Dados Aplicada: Vendas e Perfil de Clientes no Mercado de Ciclismo

- **Objetivo inicial definido**: Avaliar o comportamento e a evolução do mercado de bicicletas ao longo dos anos.

  - Perguntas a serem respondidas:
    - Quais tipos de produtos e/ou categoria movimentam mais o mercado?
    - Como o comportamento de compra varia entre diferentes perfis de clientes?
    - Em quais tipos de produtos a empresa deve investir?
    - Quais países representam os maiores mercados para a empresa?

# 1. Busca pelos Dados: 
- Este MPV tem como objetivo simular a situação de uma empresa do setor de comércio de bicicletas e acessórios esportivos que atua em diversas regiões do mundo, utilizando análise de dados para definição de estratégias para a expansão da empresa. Por fim, espera-se demonstrar o poder da engenharia e análise de dados aplicados ao suporte de decisões de negócio, em um contexto que simula os desafios reais enfrentados por empresas *data driven*.

- **Link da fonte de dados original**: https://www.kaggle.com/datasets/sadiqshah/bike-sales-in-europe


# 2. Coleta

- A etapa de coleta deste MVP se deu única e exclusivamente pelo armazenamento dos dados brutos no ambiente DBFS (Databricks File System) do databricks.

# 3. Modelagem
- A descrição detalhada com diagramas de modelagem, catálogo de dados e demais informações está no arquivo **readme** do repositório do projeto no Github.

- O modelo de dados definido para o problema foi o **SnowFlake**. Essa escolha se deu principalmente para evitar a repetição desnecessária de dados e manter a integridade, facilidade de manutenção e a escalabilidade, pensando que esses dados tendem a crescer com o tempo, tanto em dados históricos mas também em número de produtos e regiões. Além disso, apesar de envolver mais joins que um modelo em estrela, o **SnowFlake** é eficiente em ambientes de Data Warehouse modernos, como o Databricks, que são otimizados para trabalhar com tabelas normalizadas e grandes volumes de dados.

- O primeiro passo foi desenvolver a modelagem conceitual dos dados definindo a dinâmica de relacionamento das diferentes informações contidas na base de dados original. Uma particularidade desse modelo é que as informações de clientes que realizaram compra encontram-se agrupadas, ou seja, a granularidade máxima desses atributos é dada como grupos de clientes por idade e por sexo. Sendo assim, uma cidade pode conter 1 ou n grupos de clientes, assim como um grupo de clientes pode estar em uma ou n cidades. 

- Em seguida, foi feita a modelagem lógica das entidades e relacionamentos, onde foram definidas quais as colunas devem estar contidas em cada tabela e quais são as chaves primárias e extrangeiras para cruzar as informações. 

- Por fim, foram definidas as seguintas tabelas para a camada gold do Data Warehouse:
  - **FactSales**: Tabela fato de vendas 
  - **DimProducts**: Tabela dimensão de produtos
  - **DimCustomers**: Tabela dimensão de clientes
  - **DimRegion**: Tabela dimensão de regiões de venda


# 4. Carga

- A etapa de Carga compõe a carga dos dados para o Data Warehouse e o processo de ETL (Extract, Transform, Load), sendo responsável por importar, tratar e organizar os dados da base bruta, estruturando-os de maneira adequada para análises. O processo foi dividido em três camadas principais, seguindo a arquitetura em camadas (bronze, silver e gold)

- **Camada bronze**: o arquivo original da fonte de dados foi carregado diretamente para o ambiente do Databricks usando PySpark. O conteúdo foi lido como um DataFrame e salvo no formato **Delta Lake** para garantir confiabilidade nas transações e uma melhor performance das consultas.

- **Camada silver**: tratamento, limpeza, padronização e validação da qualidade dos dados presentes na camada bronze. Abaixo estão as validações de *Data Quality* que foram aplicadas nessa camada.
  - Compatibilidade dos tipos de dados por coluna
  - Verificação de Duplicatas
  - Verificação de Nulos

- **Camada gold**: nessa camada foram salvos os dados consolidados, agregados e modelados para consumo analítico, como já detalhado no item 3.

In [0]:
# Importando a base de dados
url = 'dbfs:/FileStore/tables/Sales.csv'
dataframe_raw = spark.read.csv(url, header = True, inferSchema = True)

In [0]:
%sql 
-- Criando a camada bronze
DROP DATABASE IF EXISTS bronze CASCADE;
CREATE DATABASE bronze;

In [0]:
# Salvando os dados brutos na camada bronze
dataframe_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.sales_data")

In [0]:
%sql 
-- Checando as 5 primeiras linhas da base bruta
SELECT * FROM bronze.sales_data LIMIT 5

Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue
2013-11-26,26,November,2013,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
2015-11-26,26,November,2015,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
2014-03-23,23,March,2014,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,23,45,120,1366,1035,2401
2016-03-23,23,March,2016,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,20,45,120,1188,900,2088
2014-05-15,15,May,2014,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,238,180,418


In [0]:
%sql 
-- Criando a camada silver
DROP DATABASE IF EXISTS silver CASCADE;
CREATE DATABASE silver;

In [0]:
# Importando módulos do Pyspark
from pyspark.sql.functions import col, sum

# Armazenando a tabela da camada bronze em um dataframe spark
df = spark.read.table("bronze.sales_data")

# Avaliando os tipos de dados, nomes de colunas e se valores nulos são aceitos
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Customer_Age: integer (nullable = true)
 |-- Age_Group: string (nullable = true)
 |-- Customer_Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Product_Category: string (nullable = true)
 |-- Sub_Category: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Order_Quantity: integer (nullable = true)
 |-- Unit_Cost: integer (nullable = true)
 |-- Unit_Price: integer (nullable = true)
 |-- Profit: integer (nullable = true)
 |-- Cost: integer (nullable = true)
 |-- Revenue: integer (nullable = true)



In [0]:
# Verificando se existem valores nulos na base de dados
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+----+---+-----+----+------------+---------+---------------+-------+-----+----------------+------------+-------+--------------+---------+----------+------+----+-------+
|Date|Day|Month|Year|Customer_Age|Age_Group|Customer_Gender|Country|State|Product_Category|Sub_Category|Product|Order_Quantity|Unit_Cost|Unit_Price|Profit|Cost|Revenue|
+----+---+-----+----+------------+---------+---------------+-------+-----+----------------+------------+-------+--------------+---------+----------+------+----+-------+
|   0|  0|    0|   0|           0|        0|              0|      0|    0|               0|           0|      0|             0|        0|         0|     0|   0|      0|
+----+---+-----+----+------------+---------+---------------+-------+-----+----------------+------------+-------+--------------+---------+----------+------+----+-------+



In [0]:
%sql
-- Consultando quantas duplicatas existem na base de dados
SELECT 
  COUNT(*) - COUNT(DISTINCT *) AS linhas_duplicadas
FROM bronze.sales_data;

linhas_duplicadas
1000


In [0]:
# Retirando as duplicatas e salvando a base na camada silver
df_silver = df.dropDuplicates()
df_silver.write.format("delta").mode("overwrite").saveAsTable("silver.sales_data_silver")

In [0]:
%sql 
-- Criando a camada gold
DROP DATABASE IF EXISTS gold CASCADE;
CREATE DATABASE gold;

In [0]:
%sql
-- Criando a tabela
DROP TABLE IF EXISTS gold.DimProducts;
CREATE TABLE gold.DimProducts (
    ID_Product BIGINT GENERATED ALWAYS AS IDENTITY,
    Product STRING NOT NULL,
    Category_Product STRING NOT NULL,
    SubCategory_Product STRING NOT NULL
) USING DELTA;

-- Salvando os dados na tabela DimProducts
INSERT INTO gold.DimProducts (Product, Category_Product, SubCategory_Product)
SELECT DISTINCT Product, Product_Category, Sub_Category 
FROM silver.sales_data_silver

num_affected_rows,num_inserted_rows
138,138


In [0]:
%sql
-- Criando a tabela
DROP TABLE IF EXISTS gold.DimCustomers;
CREATE TABLE gold.DimCustomers (
    ID_Customer BIGINT GENERATED ALWAYS AS IDENTITY,
    Gender STRING NOT NULL,
    Age_Group STRING NOT NULL
) USING DELTA;

-- Salvando os dados na tabela DimCustomers
INSERT INTO gold.DimCustomers (Gender, Age_Group)
SELECT DISTINCT Customer_Gender, Age_Group 
FROM silver.sales_data_silver

num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
-- Criando a tabela
DROP TABLE IF EXISTS gold.DimRegion;
CREATE TABLE gold.DimRegion (
    ID_Region BIGINT GENERATED ALWAYS AS IDENTITY,
    Country STRING NOT NULL,
    State STRING NOT NULL
) USING DELTA;

-- Salvando os dados na tabela DimCustomers
INSERT INTO gold.DimRegion (Country, State)
SELECT DISTINCT Country, State 
FROM silver.sales_data_silver

num_affected_rows,num_inserted_rows
53,53


In [0]:
%sql
-- Criando a tabela fato com os atributos modelados para a camada gold
DROP TABLE IF EXISTS gold.FactSales;

CREATE TABLE gold.FactSales (
    Sale_Date DATE NOT NULL,
    ID_Customer BIGINT,
    ID_Product BIGINT,
    ID_Region BIGINT,
    Order_Quantity INT NOT NULL,
    Unit_Cost INT NOT NULL,
    Unit_Price INT NOT NULL,
    Profit INT NOT NULL,
    Cost INT NOT NULL,
    Revenue INT NOT NULL
) USING DELTA;

-- Populando a tabela fato de vendas com os dados da tabela original armazenada na camada silver
INSERT INTO gold.FactSales (
    Sale_Date, ID_Customer, ID_Product, ID_Region,
    Order_Quantity, Unit_Cost, Unit_Price,
    Profit, Cost, Revenue
)
SELECT 
    s.Date,
    c.ID_Customer,
    p.ID_Product,
    r.ID_Region,
    s.Order_Quantity,
    s.Unit_Cost,
    s.Unit_Price,
    s.Profit,
    s.Cost,
    s.Revenue
FROM silver.sales_data_silver s
JOIN gold.DimProducts p 
  ON s.Product = p.Product 
  AND s.Product_Category = p.Category_Product 
  AND s.Sub_Category = p.SubCategory_Product
JOIN gold.DimCustomers c 
  ON s.Customer_Gender = c.Gender 
  AND s.Age_Group = c.Age_Group
JOIN gold.DimRegion r 
  ON s.Country = r.Country 
  AND s.State = r.State;

-- Visualizando as 3 primeiras linhas da base de dados
SELECT * FROM gold.FactSales LIMIT 5;


Sale_Date,ID_Customer,ID_Product,ID_Region,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue
2016-02-22,2,44,22,16,13,35,341,208,549
2013-12-13,7,112,31,3,13,35,44,39,83
2015-08-14,1,112,45,27,13,35,528,351,879
2016-03-19,2,44,24,24,13,35,494,312,806
2015-10-22,3,112,9,15,13,35,215,195,410


# 5. Análise

## 5.a Data Quality
- Considerando que os atributos de todas as tabelas desse modelo derivam da tabela original do projeto, a discussão de data quality será feita com base nas colunas da tabela original.

#### 1. Valores Nulos: 
A fonte de dados utilizada para esse projeto já estava relativamente limpa e estruturada, de modo que não existiam valores nulos em nenhuma das colunas **(etapa detalhada na camada silver)**.

#### 2. Valores Duplicados
Em relação a valores duplicados, foram encontrados 1000 registros duplicados na base de dados e a trativa escolhida foi simplesmente excluir a duplicidade de dados **(etapa detalhada na camada silver)** para não haver influência nas análises.

#### 3. Consistência
- A coluna **Sale_Date** está formatada como data (yyyy-MM-dd) e suas derivadas Day, Month_Name e Year apresentam-se no formato esperado

- Variáveis categóricas: **Customer_Gender**, **Age_Group**, **Product**, **Country**, **State**, **Product_Category**, **Sub_Category** não contém erros de digitação ou inconsistências (ex: "F" vs "Female")

- Variáveis numéricas: **Order_Quantity**, **Unit_Cost**, **Unit_Price**, **Profit**, **Cost** e **Revenue** não apresentam valores negativos ou incoerentes

## 5.b Solução do problema proposto
Seguem nessa seção as análises e respostas paras as perguntas definidas previamente nos objetivos do trabalho

- **Quais tipos de produtos e/ou categoria movimentam mais o mercado?**

  - **Resposta**: A análise dos TOP 10 produtos em vendas, considerando tanto unidades quanto faturamento, revela dois comportamentos distintos no mercado. Em termos de volume, as categorias de Acessórios e Roupas lideram com folga, refletindo a alta rotatividade e o caráter mais consumível desses itens.

    Por outro lado, quando o critério é faturamento, a categoria de Bicicletas se destaca — o que era esperado, já que esses produtos têm um ticket médio significativamente mais alto, apesar do menor volume de vendas.

    Esse contraste entre volume e valor evidencia a importância de estratégias diferenciadas por categoria: enquanto acessórios e roupas impulsionam a frequência de compra e fidelização, as bicicletas são responsáveis por uma parcela expressiva da receita.

In [0]:
%sql
-- Top 10 produtos/categorias mais vendidos em volume
SELECT
    dp.Category_Product,
    dp.SubCategory_Product,
    SUM(fs.Order_Quantity) AS total_quantity
FROM gold.FactSales fs
JOIN gold.DimProducts dp 
  ON fs.ID_Product = dp.ID_Product
GROUP BY 
    dp.Category_Product,
    dp.SubCategory_Product
ORDER BY 
    total_quantity DESC
LIMIT 10;

Category_Product,SubCategory_Product,total_quantity
Accessories,Tires and Tubes,505889
Accessories,Bottles and Cages,238610
Accessories,Helmets,181522
Clothing,Jerseys,88095
Clothing,Caps,67268
Accessories,Fenders,62138
Clothing,Gloves,39978
Accessories,Cleaners,27574
Clothing,Shorts,27168
Accessories,Hydration Packs,19914


In [0]:
%sql
-- Top 10 produtos/categorias mais relevantes em faturamento
SELECT
    dp.Category_Product,
    dp.SubCategory_Product,
    SUM(fs.Revenue) AS total_revenue
FROM gold.FactSales fs
JOIN gold.DimProducts dp 
  ON fs.ID_Product = dp.ID_Product
GROUP BY 
    dp.Category_Product,
    dp.SubCategory_Product
ORDER BY 
    total_revenue DESC
LIMIT 10;

Category_Product,SubCategory_Product,total_revenue
Bikes,Road Bikes,33150708
Bikes,Mountain Bikes,21020794
Bikes,Touring Bikes,7262982
Accessories,Helmets,5738482
Accessories,Tires and Tubes,4598124
Clothing,Jerseys,4112382
Clothing,Shorts,1740710
Accessories,Bottles and Cages,1390570
Accessories,Fenders,1245733
Accessories,Hydration Packs,990406


- **Como o comportamento de compra varia entre diferentes perfis de clientes?**
  - **Resposta**: Tradicionalmente, o ciclismo é um esporte dominado pelo público masculino — tanto em número de praticantes quanto na variedade de produtos disponíveis. Isso muitas vezes leva as mulheres a utilizarem equipamentos desenvolvidos para homens.

    Entretanto, as três *queries* abaixo revelam uma mudança nesse cenário: há uma tendência clara de equiparação no comportamento de compra entre os gêneros.

    Primeiro, ao analisar as vendas anuais de bicicletas por gênero, observa-se um volume de unidades vendidas bastante semelhante entre homens e mulheres. Em seguida, os dados de faturamento por grupo de clientes e o perfil de consumo também indicam uma equivalência consistente.

    Esse panorama destaca uma oportunidade estratégica: investir no desenvolvimento de um portfólio direcionado ao público feminino pode fortalecer a presença da marca nesse segmento e impulsionar o crescimento, atendendo a uma demanda cada vez mais expressiva.

In [0]:
%sql
-- Calulando o histórico de vendas por gênero
SELECT 
    YEAR(f.Sale_Date) AS Ano,
    c.Gender AS Genero,
    ROUND(SUM(f.Order_Quantity), 2) AS Units_Sold
FROM gold.FactSales f
JOIN gold.DimCustomers c ON f.ID_Customer = c.ID_Customer
JOIN gold.DimProducts p ON f.ID_Product = p.ID_Product
WHERE p.Category_Product = 'Bikes'
GROUP BY 
    YEAR(f.Sale_Date),
    c.Gender
ORDER BY Ano, Genero;

Ano,Genero,Units_Sold
2011,F,2689
2011,M,2565
2012,F,2736
2012,M,2592
2013,F,2852
2013,M,2798
2014,F,2140
2014,M,2412
2015,F,4290
2015,M,4230


In [0]:
%sql
-- Faturamento por grupo de clientes
SELECT 
    dc.Gender,
    dc.Age_Group,
    SUM(fs.Revenue) AS total_revenue
FROM gold.FactSales fs
JOIN gold.DimCustomers dc 
  ON fs.ID_Customer = dc.ID_Customer
GROUP BY 
    dc.Gender,
    dc.Age_Group
ORDER BY 
    total_revenue DESC;

Gender,Age_Group,total_revenue
M,Adults (35-64),21207395
F,Adults (35-64),21159786
M,Young Adults (25-34),15391679
F,Young Adults (25-34),15075353
M,Youth (<25),6355830
F,Youth (<25),5329659
M,Seniors (64+),177179
F,Seniors (64+),129891


In [0]:
%sql
-- Perfil de compra para cada grupo
SELECT 
    c.Gender,
    c.Age_Group,
    p.SubCategory_Product,
    SUM(f.Order_Quantity) AS total_orders,
    SUM(f.Revenue) AS total_revenue,
    ROUND(AVG(f.Revenue), 2) AS avg_price
FROM gold.FactSales f
JOIN gold.DimCustomers c ON f.ID_Customer = c.ID_Customer
JOIN gold.DimProducts p ON f.ID_Product = p.ID_Product
GROUP BY c.Gender, c.Age_Group, p.SubCategory_Product
ORDER BY total_revenue DESC
LIMIT 10;

Gender,Age_Group,SubCategory_Product,total_orders,total_revenue,avg_price
F,Adults (35-64),Road Bikes,4718,8101329,2534.83
M,Adults (35-64),Road Bikes,4503,7520714,2441.0
M,Young Adults (25-34),Road Bikes,3811,6366190,2419.68
F,Young Adults (25-34),Road Bikes,3564,5980321,2461.04
F,Adults (35-64),Mountain Bikes,2976,5251065,2335.88
M,Adults (35-64),Mountain Bikes,2818,5229238,2453.89
F,Young Adults (25-34),Mountain Bikes,2346,4172327,2451.43
M,Young Adults (25-34),Mountain Bikes,2191,3643556,2288.67
M,Youth (<25),Road Bikes,1697,2774659,2495.2
F,Youth (<25),Road Bikes,1309,2348012,2798.58


- **Em quais tipos de produtos a empresa deve investir?**
  - **Resposta**: As *queries* abaixo demonstram que, embora a venda de bicicletas gere o maior lucro líquido total — devido ao alto ticket médio —, a margem de lucro percentual dessa categoria é uma das mais baixas. Isso indica que o retorno sobre o investimento não é tão vantajoso quanto parece à primeira vista.

    Por outro lado, categorias como vestuário e acessórios (ex: meias, para-lamas, bermudas) apresentam margens de lucro mais altas. Além disso, esses produtos são comprados com mais frequência e têm ciclo de reposição mais curto, o que favorece maior rotatividade e fidelização dos clientes.

    Dessa forma, a empresa deve considerar direcionar seus investimentos para essas categorias, priorizando itens de maior margem e maior frequência de compra.

In [0]:
%sql
-- Lucro líquido por categoria
SELECT
    dp.Category_Product,
    dp.SubCategory_Product,
    SUM(fs.Profit) AS total_profit
FROM gold.FactSales fs
JOIN gold.DimProducts dp 
  ON fs.ID_Product = dp.ID_Product
GROUP BY 
    dp.Category_Product,
    dp.SubCategory_Product
ORDER BY 
    total_profit DESC
LIMIT 10;

Category_Product,SubCategory_Product,total_profit
Bikes,Road Bikes,10012631
Bikes,Mountain Bikes,8117801
Accessories,Helmets,3378696
Accessories,Tires and Tubes,2685467
Bikes,Touring Bikes,2269294
Clothing,Shorts,1034342
Accessories,Bottles and Cages,799882
Accessories,Fenders,748914
Clothing,Jerseys,603440
Accessories,Hydration Packs,572668


In [0]:
%sql
-- Margem de lucro por categoria
SELECT 
    p.Category_Product,
    p.SubCategory_Product,
    ROUND(SUM(f.Profit) * 100.0 / SUM(f.Revenue), 2) AS Margem_Percentual_Media
FROM gold.FactSales f
JOIN gold.DimProducts p ON f.ID_Product = p.ID_Product
GROUP BY p.Category_Product, p.SubCategory_Product
ORDER BY Margem_Percentual_Media DESC;

Category_Product,SubCategory_Product,Margem_Percentual_Media
Clothing,Socks,63.31
Accessories,Fenders,60.12
Clothing,Shorts,59.42
Accessories,Helmets,58.88
Accessories,Bike Racks,58.80
Clothing,Gloves,58.72
Accessories,Bike Stands,58.69
Accessories,Tires and Tubes,58.40
Accessories,Cleaners,58.29
Accessories,Hydration Packs,57.82


- **Quais países representam os maiores mercados para a empresa?**
  - **Resposta**: A *query* a seguir revela que os principais mercados da empresa atualmente estão fora da Europa, com Estados Unidos e Austrália concentrando a maior parte do faturamento.

    Apesar desse bom desempenho internacional, é sabido que a Europa é, de longe, o maior mercado global para o ciclismo. Isso evidencia uma grande oportunidade de expansão para a empresa no continente europeu, onde sua presença ainda está abaixo do potencial de mercado.

    Investir em estratégias regionais de marketing, logística e portfólio adaptado ao perfil do consumidor europeu pode permitir à empresa ganhar representatividade em um mercado altamente promissor e alinhado ao seu setor de atuação.

In [0]:
%sql
SELECT 
    r.Country,
    SUM(f.Revenue) AS Total_Revenue
FROM gold.FactSales f
JOIN gold.DimRegion r
  ON f.ID_Region = r.ID_Region
GROUP BY r.Country
ORDER BY Total_Revenue DESC;

Country,Total_Revenue
United States,27777098
Australia,21196395
United Kingdom,10575628
Germany,8956724
France,8414745
Canada,7906182
